<h3>Exploring biobank data</h3>
<p>
    This document aims to amswer the first Basic Questions:    
    <ul style="list-style-type:circle">
      <li>How many samples are there?</li>
      <li>How many serum?</li>
      <li>What is the average age at sampling?</li>
      <li>Are there more male DNA sample than expected by chance? (use Fisher’s exact)</li>
    </ul>
</p>

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import fisher_exact

In [2]:
# load the tables
data = pd.read_csv('data/sample_catalogue_rd_connect.csv')
sex = pd.read_csv('data/rdconnect_Sex_2017-10-04_02-52-42.csv')
disease = pd.read_csv('data/rdconnect_Disease_2017-10-04_02-59-41.csv')
materialtype = pd.read_csv('data/rdconnect_MaterialType_2017-10-04_02-53-27.csv')
anatomicalsite = pd.read_csv('data/rdconnect_AnatomicalSite_2017-10-04_02-56-42.csv')

In [3]:
disease.head(10)

,IRI,PreferredTerm,Description,Code
0,http://www.orpha.net/ORDO/Orphanet_1440,Ring chromosome 14,Ring chromosome 14,1440
1,http://identifiers.org/ncit/C49651,Healthy Subject,An individual who is or becomes a participant ...,C49651
2,urn:miriam:orphanet:Orphanet_802,Multiple sclerose,NaN,802
3,urn:miriam:orphanet:Orphanet_424925,Qualitative or quantitative defects of Torsin-...,NaN,424925
4,urn:miriam:orphanet:Orphanet_424933,Rare epithelial tumor of liver and intrahepati...,NaN,424933
5,urn:miriam:orphanet:Orphanet_424936,Carcinoma of liver and intrahepatic biliary tract,NaN,424936
6,urn:miriam:orphanet:Orphanet_424943,Adenocarcinoma of liver and intrahepatic bilia...,NaN,424943
7,urn:miriam:orphanet:Orphanet_424970,Undifferentiated carcinoma of liver and intrah...,NaN,424970
8,urn:miriam:orphanet:Orphanet_424975,Squamous cell carcinoma of liver and intrahepa...,NaN,424975
9,urn:miriam:orphanet:Orphanet_424982,Biliary cystadenocarcinoma,NaN,424982


In [4]:
data.head()

,Disease,MaterialType,MaterialTypeDetailed,AnatomicalSite,Sex,DiagnosisType,Genotype,AgeAtSampling,Affected,Family,RelatedSamples,Registry,BiobankID,RegistryID
0,urn:miriam:ncit:C46951,SERUM,http://purl.obolibrary.org/obo/FMA_63083,http://purl.obolibrary.org/obo/UBERON_0000178,http://purl.obolibrary.org/obo/PATO_0000384,CLINICAL,FALSE,15.0,TRUE,UNK,NAV,UNK,87919,NaN
1,urn:miriam:ncit:C46951,SERUM,http://purl.obolibrary.org/obo/FMA_63083,http://purl.obolibrary.org/obo/UBERON_0000178,http://purl.obolibrary.org/obo/PATO_0000384,CLINICAL,FALSE,20.0,TRUE,UNK,NAV,UNK,87919,NaN
2,urn:miriam:orphanet:Orphanet_101075,OTHER,http://purl.obolibrary.org/obo/FMA_63877,http://purl.obolibrary.org/obo/UBERON_0002097,http://purl.obolibrary.org/obo/PATO_0000384,MOLECULAR,TRUE,26.0,TRUE,UNK,UNK,UNK,87919,NaN
3,urn:miriam:orphanet:Orphanet_101075,OTHER,http://purl.obolibrary.org/obo/FMA_63877,http://purl.obolibrary.org/obo/UBERON_0002097,http://purl.obolibrary.org/obo/PATO_0000384,MOLECULAR,TRUE,26.0,TRUE,UNK,UNK,UNK,87919,NaN
4,urn:miriam:orphanet:Orphanet_101081,SERUM,http://purl.obolibrary.org/obo/FMA_63083,http://purl.obolibrary.org/obo/UBERON_0000178,http://purl.obolibrary.org/obo/PATO_0000384,MOLECULAR,TRUE,19.0,TRUE,UNK,UNK,UNK,87919,NaN


In [5]:
sex.head()

,IRI,Preferred term,Description,Code
0,http://purl.obolibrary.org/obo/PATO_0000383,Female,Female,383
1,http://purl.obolibrary.org/obo/PATO_0000384,Male,Male,384
2,http://purl.obolibrary.org/obo/PATO_0001340,Ambiguous,A biological sex quality\n inhering in an org...,1340
3,https://github.com/MIABIS/miabis/wiki/Structur...,Unknown,A proper value is applicable but not known,0


'Male'

Sex
Female     2457
Male       3850
Unknown    1045
Name: Genotype, dtype: int64

Genotype
FALSE    4178
TRUE     2795
UNK       379
Name: Sex, dtype: int64

(1422, 14)

Genotype  False  True 
Sex                   
False      2533    969
True       2024   1826


(2.3583142639206711, 1.1059640270817867e-68)

In [10]:
print "There are %d samples in total."%data.shape[0]

There are 7352 samples in total.


In [21]:
print "There are %d serum in total."%len(data[data.MaterialType=='SERUM'])

There are 2148 serum in total.


In [37]:
print "The average age for sampling is %d."%data.AgeAtSampling.mean()

The average age for sampling is 38.


In [6]:
# whether there are more male DNA samples than expected?
def convert_sex(sex_from_data):
    return sex['Preferred term'][sex['IRI']==sex_from_data].values[0]
data['Sex'] = data.Sex.apply(convert_sex)
data.groupby(by='Sex').count().Genotype
data.groupby(by='Genotype').count().Sex
# data.loc[(data.Sex=='Female') & (data.Genotype=='FALSE')].shape
# including Unknown Sex and UNK for DN samples ?
tab = pd.crosstab(data.Sex=='Male',data.Genotype=='TRUE')
print(tab)
if fisher_exact(tab)[1] <0.05:
    print("yes, there are more Male DNA samples than expected.")
else:
    print("no, there are not more male DNA sample than expected.")

Genotype  False  True 
Sex                   
False      2533    969
True       2024   1826
yes, there are more Male DNA samples than expected.
